In [1]:
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn.functional as F
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import torch_geometric.transforms as T
from torch_geometric.data import DataLoader
from torch_geometric.nn import SAGEConv

In [2]:
dataset = PygNodePropPredDataset(name='ogbn-arxiv', 
                                 root='../data/dataset/',
                                 transform=T.ToSparseTensor())

In [3]:
torch.cuda.is_available()

True

In [4]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

In [5]:
data = dataset[0]

In [6]:
data

Data(adj_t=[169343, 169343, nnz=1166243], node_year=[169343, 1], x=[169343, 128], y=[169343, 1])

In [7]:
data = data.to(device)

In [8]:
split_idx = dataset.get_idx_split()
train_idx = split_idx['train'].to(device)

In [9]:
evaluator = Evaluator(name='ogbn-arxiv')

In [10]:
data['x'].shape[1]

128

In [11]:
data['y'][:, 0].unique().shape[0]

40

In [12]:
n_iters = 3000
epochs = 10000
log_steps = 100
input_dim = data['x'].shape[1]
hidden_channels = input_dim * 2
output_dim = data['y'][:, 0].unique().shape[0]
lr_rate = 0.001

In [13]:
class OneLayer(torch.nn.Module):
    def __init__(self, input_dim, hidden_channels, output_dim):
        super(OneLayer, self).__init__()
        
        self.convs = torch.nn.ModuleList()
        self.bns = torch.nn.ModuleList()
        
        
        self.convs.append(SAGEConv(input_dim, output_dim))
        

    def forward(self, x, adj_t):
        x = self.convs[0](x, adj_t)
        return x.log_softmax(dim=-1)

In [14]:
model = OneLayer(input_dim, hidden_channels, output_dim).to(device)

In [15]:
def train(model, data, train_idx, optimizer):
    model.train()

    optimizer.zero_grad()
    out = model(data.x, data.adj_t)[train_idx]
    loss = F.nll_loss(out, data.y.squeeze(1)[train_idx])
    loss.backward()
    optimizer.step()

    return loss.item()

In [16]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    model.eval()

    out = model(data.x, data.adj_t)
    y_pred = out.argmax(dim=-1, keepdim=True)

    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr_rate)
for epoch in range(1, 1 + epochs):
    loss = train(model, data, train_idx, optimizer)
    result = test(model, data, split_idx, evaluator)

    if epoch % log_steps == 0:
        train_acc, valid_acc, test_acc = result
        print(f'Epoch: {epoch:02d}, '
              f'Loss: {loss:.4f}, '
              f'Train: {100 * train_acc:.2f}%, '
              f'Valid: {100 * valid_acc:.2f}% '
              f'Test: {100 * test_acc:.2f}%')

Epoch: 100, Loss: 2.6869, Train: 27.51%, Valid: 27.36% Test: 26.63%
Epoch: 200, Loss: 2.3460, Train: 37.84%, Valid: 38.77% Test: 34.62%
Epoch: 300, Loss: 2.1141, Train: 45.13%, Valid: 45.47% Test: 39.42%
Epoch: 400, Loss: 1.9518, Train: 49.12%, Valid: 49.00% Test: 42.86%
Epoch: 500, Loss: 1.8343, Train: 51.68%, Valid: 51.18% Test: 45.06%
Epoch: 600, Loss: 1.7467, Train: 53.33%, Valid: 52.86% Test: 46.74%
Epoch: 700, Loss: 1.6793, Train: 54.49%, Valid: 54.03% Test: 47.93%
Epoch: 800, Loss: 1.6263, Train: 55.43%, Valid: 54.71% Test: 48.83%
Epoch: 900, Loss: 1.5836, Train: 56.16%, Valid: 55.15% Test: 49.50%
Epoch: 1000, Loss: 1.5486, Train: 56.75%, Valid: 55.71% Test: 50.01%
Epoch: 1100, Loss: 1.5194, Train: 57.26%, Valid: 56.20% Test: 50.51%
Epoch: 1200, Loss: 1.4948, Train: 57.75%, Valid: 56.62% Test: 50.91%
Epoch: 1300, Loss: 1.4737, Train: 58.19%, Valid: 56.90% Test: 51.21%
Epoch: 1400, Loss: 1.4555, Train: 58.53%, Valid: 57.22% Test: 51.53%
Epoch: 1500, Loss: 1.4396, Train: 58.82%, V